# Библиотеки

In [178]:
import pandas as pd
import math
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score

# Откроем и прочитаем файл

In [179]:
data = pd.read_csv('data-logistic.csv', header=None)
data

,0,1,2
0,-1,-0.663827,-0.138526
1,1,1.994596,2.468025
2,-1,-1.247395,0.749425
3,1,2.309374,1.899836
4,1,0.849143,2.407750
...,...,...,...
200,1,4.245176,3.053931
201,1,2.437935,1.357804
202,-1,-1.876638,1.533398
203,1,-6.824446,-13.934211


# Раздели на признаки и ответы

In [180]:
Y = data[0]
X = np.array([data[1],data[2]])

## Нарисуем целевую выборку

In [181]:
# plt.scatter(X[0, Y == 1], X[1, Y == 1], c='green', label='+1')
# plt.scatter(X[0, Y == -1], X[1, Y == -1], c='red', label='-1')
# plt.xlabel("")
# plt.ylabel("")
# plt.title('Целевая выборка')
# plt.legend();

# Формула для весов в градиентном спуске

### Способ 1

In [182]:
def fw1(w1, w2, Y, X, k, C):
    S = 0 
    l = len(Y)
    for i in range(0, l):
        S += float(Y[i]) * X[0][i] * ( 1.0  - 1.0 / ( 1.0 + np.exp(- float(Y[i]) * (w1 * X[0][i] + w2 * X[1][i]) ) ) )
    return w1 + k * S / l - k * C * w1

In [183]:
def fw2(w1, w2, Y, X, k, C):
    S = 0 
    l = len(Y)
    for i in range(0, l):
        S += float(Y[i]) * X[1][i] * ( 1.0  - 1.0 / ( 1.0 + np.exp(- float(Y[i]) * (w1 * X[0][i] + w2 * X[1][i]) ) ) )
    return w2 + k * S / l - k * C * w2

### Способ 2

In [184]:
def gradstep(l, xi1, xi2, k, yi, C, w1, w2):
    w1 = w1 + k * np.mean(yi * xi2 * (1 - 1 / (1 + np.exp(-yi *(w1 * x1 + w2 * x2))))) - k * C * w1
    w2 = w2 + k * np.mean(yi * xi2 * (1 - 1 / (1 + np.exp(-yi *(w1 * x1 + w2 * x2))))) - k * C * w2
    return w1, w2

# Реализация градиентного спуска для обычной и L2 - регуляризованной цепи

In [185]:
# k - длина шага
# X и Y соответственно признаки и ответы
# w1 и w2 - которые равны 0 - начальные веса
# err - евклидово расстояние между векторами весов должно быть не больше этотго значения

In [186]:
def grad(Y, X, C = 0.0, w1 = 0.0, w2 = 0.0, k = 0.1, err = 1e-5):
    i = 0
    i_max = 1000000
    w1_new, w2_new = w1, w2
    
    while(True):
        i += 1
        w1_new, w2_new = fw1(w1, w2, Y, X, k, C), fw2(w1, w2, Y, X, k, C)
        e = math.sqrt((w1_new - w1) ** 2 + (w2_new - w2) ** 2)
        if i >= i_max or e <= err:
            break
        else:
            w1, w2 = w1_new, w2_new
 
    return [w1_new, w2_new]

# Запуск градиентного спуска

In [187]:
w1, w2 = grad(Y, X)
rw1, rw2 = grad(Y, X, 10.0)

In [188]:
def a(X, w1, w2):
    return 1.0 / (1.0 + math.exp(- w1 * X[0] - w2 * X[1]))

In [189]:
X = pd.DataFrame(X.transpose())

In [190]:
y_score = X.apply(lambda x: a(x, w1, w2), axis=1)
y_rscore = X.apply(lambda x: a(x, rw1, rw2), axis=1)

In [191]:
print(roc_auc_score(Y, y_score))
print(roc_auc_score(Y, y_rscore))

0.9268571428571428
0.9362857142857142
